In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



2024-04-03 10:39:04.940410: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 10:39:04.972236: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 10:39:04.972262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 10:39:04.973087: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-03 10:39:04.978408: I tensorflow/core/platform/cpu_feature_guar

In [6]:
# import wandb
# import tensorflow as tf
# import numpy as np
# import h5py
# from tensorflow.keras import layers, models, optimizers

# # Ensure wandb is installed: pip install wandb
# # Initialize your W&B project before running the script

# # Dataset loading and preparation
# def load_and_prepare_dataset(batch_size):
#     with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
#         images = np.array(hdf.get('images'))
#         boxes = np.array(hdf.get('boxes'))
    

#     image_normalized = (images + 1e-9) / 9.26
#     normalized_boxes = boxes / [1, 64, 64, 64, 64]

#     images_np = image_normalized
#     probabilities = np.array(normalized_boxes[:, :, :-4])
#     probabilities = tf.expand_dims(probabilities, axis=1)
#     boxes_np = np.array(normalized_boxes[:, :, 1:])
#     boxes_np = tf.expand_dims(boxes_np, axis=1)

#     dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
#     dataset = dataset.shuffle(buffer_size=8000).batch(batch_size)
#     return dataset

# # Model building
# def build_model():
#     input_shape = (64, 64, 1)
#     num_classes = 280
#     num_coordinates = 4

#     x_input = layers.Input(shape=input_shape)
#     #Layer 1
#     x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
#     x = layers.MaxPool2D()(x)
#     x = layers.BatchNormalization()(x) 
    
#     x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
#     #Layer 2
#     x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
#     x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
#     #Layer 3
#     x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
#     x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
#     #Layer 4
#     x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
#     x = layers.MaxPool2D()(x)
#     #Layer 5
#     x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
#     x = layers.MaxPool2D()(x)
#     x = layers.BatchNormalization()(x) 
    
    
#     x = layers.Flatten()(x)
#     x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
#     x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
#     x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
#     x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

#     model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
#     return model

# # Optimizer selection
# def select_optimizer(optimizer_name, learning_rate):
#     if optimizer_name == 'adam':
#         return optimizers.Adam(learning_rate=learning_rate)
#     elif optimizer_name == 'sgd':
#         return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
#     else:
#         raise ValueError("Unsupported optimizer specified.")

# # Training function with sweep
# def train_with_sweep():
    
#     with wandb.init():
#         config = wandb.config
#         # # config.run.id
#         # print(wandb.run.id)
#         # print(wandb.run.name)
#         dataset = load_and_prepare_dataset(config.batch_size)
#         model = build_model()
        
#         optimizer = select_optimizer(config.optimizer, config.learning_rate)
#         model.compile(optimizer=optimizer, 
#                       loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
#                       metrics={'x_prob_reshape': 'accuracy'})

#         # Fit the model
#         model.fit(dataset, epochs=config.epochs, callbacks=[wandb.keras.WandbCallback()])
#         # model.save(f'{wandb.run.name}.h5')


In [2]:
import wandb
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers, models, optimizers

# Ensure wandb is installed: pip install wandb
# Initialize your W&B project before running the script

# Dataset loading and preparation
def load_and_prepare_dataset(batch_size):
    with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
        images = np.array(hdf.get('images'))
        boxes = np.array(hdf.get('boxes'))
    

    image_normalized = (images + 1e-9) / 9.26
    normalized_boxes = boxes / [1, 64, 64, 64, 64]

    images_np = image_normalized
    probabilities = np.array(normalized_boxes[:, :, :-4])
    probabilities = tf.expand_dims(probabilities, axis=1)
    boxes_np = np.array(normalized_boxes[:, :, 1:])
    boxes_np = tf.expand_dims(boxes_np, axis=1)

    dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
    # dataset = dataset.shuffle(buffer_size=8000).batch(batch_size)
    
    
    
    
    total_items = 10000
    train_size = int(total_items * 0.8)
    val_size = total_items - train_size

    # Splitting the dataset
    train_dataset = dataset.take(train_size)
    val_dataset = dataset.take(val_size)


    train_dataset = train_dataset.shuffle(buffer_size=train_size,reshuffle_each_iteration=True)
    train_dataset = train_dataset.batch(batch_size)
    val_dataset = val_dataset.batch(batch_size) 
    
    
    
    
    
    return train_dataset, val_dataset

    

# Model building
def build_model():
    input_shape = (64, 64, 1)
    num_classes = 280
    num_coordinates = 4

    x_input = layers.Input(shape=input_shape)
    #Layer 1
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 2
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 3
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 4
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    #Layer 5
    x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    
    x = layers.Flatten()(x)
    x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
    x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
    x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
    x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

    model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
    return model

# Optimizer selection
def select_optimizer(optimizer_name, learning_rate):
    if optimizer_name == 'adam':
        return optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer specified.")

def train_with_sweep():
    with wandb.init():
        config = wandb.config
        # Load the dataset and unpack the training and validation datasets
        train_dataset, val_dataset = load_and_prepare_dataset(config.batch_size)
        
        # Build the model
        model = build_model()
        
        # Select the optimizer
        optimizer = select_optimizer(config.optimizer, config.learning_rate)
        
        # Compile the model
        model.compile(optimizer=optimizer, 
                      loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
                      metrics={'x_prob_reshape': 'accuracy'})
        
        # Fit the model with both training and validation datasets
        model.fit(train_dataset, 
                  epochs=config.epochs, 
                  validation_data=val_dataset, 
                  callbacks=[wandb.keras.WandbCallback()])

        model.save(f"/home/m3-learning/Documents/Research Data/Derrick's Object Detection/Models/Sweep Weights/Train and Val/{wandb.run.name}.h5")


In [5]:

sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 800}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'values': [32, 64, 256]
      }
    })

In [6]:
sweep_id = wandb.sweep(sweep_config, project="Working Hyperparameter Tuning Yichen edit A2")
wandb.agent(sweep_id, train_with_sweep, count=7)

Create sweep with ID: r213dktk
Sweep URL: https://wandb.ai/alphanium/Working%20Hyperparameter%20Tuning%20Yichen%20edit%20A2/sweeps/r213dktk


wandb: Agent Starting Run: 0lfw7t52 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 800
wandb: 	learning_rate: 0.0947534994294077
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/800
250/250 [==============================] - ETA: 0s - loss: 0.5236 - x_prob_reshape_loss: 0.4500 - x_boxes_reshape_loss: 0.0736 - x_prob_reshape_accuracy: 0.8188

/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: /home/m3-learning/wandb/run-20240403_115814-0lfw7t52/files/model-best/assets


...

INFO:tensorflow:Assets written to: /home/m3-learning/wandb/run-20240403_202709-wvvpwkmy/files/model-best/assets


INFO:tensorflow:Assets written to: /home/m3-learning/wandb/run-20240403_202709-wvvpwkmy/files/model-best/assets
wandb: Adding directory to artifact (/home/m3-learning/wandb/run-20240403_202709-wvvpwkmy/files/model-best)... Done. 0.2s


250/250 [==============================] - 8s 33ms/step - loss: 4.9670e-04 - x_prob_reshape_loss: 3.0804e-05 - x_boxes_reshape_loss: 4.6590e-04 - x_prob_reshape_accuracy: 1.0000 - val_loss: 6.7184e-05 - val_x_prob_reshape_loss: 1.7829e-05 - val_x_boxes_reshape_loss: 4.9355e-05 - val_x_prob_reshape_accuracy: 1.0000


/home/m3-learning/anaconda3/envs/tiny_yolo/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▆▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_x_boxes_reshape_loss,█▇▆▅▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_x_prob_reshape_accuracy,▁▆▇█████████████████████████████████████
val_x_prob_reshape_loss,█▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
x_boxes_reshape_loss,██▇▇▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
x_prob_reshape_accuracy,▁▄▆█████████████████████████████████████
x_prob_reshape_loss,█▅▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,799
best_val_loss,7e-05


In [ ]:
# def visualize_bounding_boxes(image, probability_vector, bounding_box_coordinates, threshold=0.9):
#     """
#     Visualizes bounding boxes on an image based on a probability vector.

#     Parameters:
#     - image: A 3D tensor representing the image.
#     - probability_vector: A 1D tensor representing the probabilities associated with bounding boxes.
#     - bounding_box_coordinates: A 2D tensor representing bounding box coordinates.
#     - threshold: Probability threshold for visualization.

#     Returns:
#     None (displays the image with bounding boxes).
#     """
#     # Convert to NumPy arrays for easier handling
#     image_np = image
#     prob_vector_np = probability_vector
#     bbox_coordinates_np = bounding_box_coordinates
#    # Denormalize image if necessary (adjust based on your normalization method)
#     denormalized_image = image_np  # Modify if normalization was applied during training

#     # Visualize the image
#     plt.figure(figsize=(4, 4))
#     plt.imshow(denormalized_image, cmap='gray')
#     plt.title("Bounding Box Visualization")

#     # Plot bounding boxes based on probability threshold
#     for i in range(len(prob_vector_np)):
#         prob = prob_vector_np[i]
#         bbox = bbox_coordinates_np[i]
#         if prob > threshold:
#             # Denormalize bounding box coordinates if necessary
#             denormalized_bbox = bbox  # Modify if normalization was applied during training
#             y1, x1, y2, x2 = denormalized_bbox
#             plt.plot([x1, x2, x2, x1, x1],[y1, y1, y2, y2, y1],
                     
#                      color='r', linewidth=2, label='Bounding Box')

   






   
#     plt.show()

# t = np.random.randint(0,400)


# visualize_bounding_boxes(tf.convert_to_tensor(inputs[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(output[1][t,0,:,:])*[64,64,64,64]) ##myprediction
# visualize_bounding_boxes(tf.convert_to_tensor(image_normalized[t]), probabilities.numpy()[t].squeeze(), tf.convert_to_tensor(boxes_np[t,0,:,:])*[64,64,64,64]) ##myprediction





In [7]:
import matplotlib.pyplot as plt

def visualize_comparisons(image, true_probs, true_boxes, pred_probs, pred_boxes, threshold=0.9):
    """
    Visualizes true and predicted bounding boxes on an image.

    Parameters:
    - image: The image to visualize.
    - true_probs: True probabilities for each bounding box.
    - true_boxes: True bounding box coordinates.
    - pred_probs: Predicted probabilities for each bounding box.
    - pred_boxes: Predicted bounding box coordinates.
    - threshold: Probability threshold for showing bounding boxes.
    """
    # Assuming image normalization that can be directly visualized
    plt.imshow(image.squeeze(), cmap='gray')  # Adjust if your image has more than one channel
    plt.title("True vs. Predicted Bounding Boxes")

    # Iterate over true boxes
    for i, prob in enumerate(true_probs):
        if prob > threshold:
            bbox = true_boxes[i]
            plt.plot([bbox[1], bbox[3], bbox[3], bbox[1], bbox[1]],
                     [bbox[0], bbox[0], bbox[2], bbox[2], bbox[0]],
                     color='green', linewidth=2, label='True Box' if i == 0 else "")

    # Iterate over predicted boxes
    for i, prob in enumerate(pred_probs):
        if prob > threshold:
            bbox = pred_boxes[i]
            plt.plot([bbox[1], bbox[3], bbox[3], bbox[1], bbox[1]],
                     [bbox[0], bbox[0], bbox[2], bbox[2], bbox[0]],
                     color='red', linewidth=2, linestyle='dashed', label='Predicted Box' if i == 0 else "")

    plt.legend()
    plt.show()


# Assuming `val_dataset` is already defined and batched
for images, labels in val_dataset.take(1):  # Take 1 batch
    # Predict on the batch
    pred_probs, pred_boxes = model.predict(images)

    # Convert predictions and labels to the required format if necessary
    # This step depends on your data and model output format

    # Visualize the first image in the batch as an example
    image = images[0].numpy()  # Assuming TensorFlow dataset
    true_probs = labels['x_prob_reshape'][0].numpy().squeeze()
    true_boxes = labels['x_boxes_reshape'][0].numpy().squeeze() * [64,64,64,64]  # Rescale if your boxes were normalized
    pred_probs = pred_probs[0].squeeze()
    pred_boxes = pred_boxes[0].squeeze() * [64,64,64,64]  # Adjust scaling to match your data preparation

    visualize_comparisons(image, true_probs, true_boxes, pred_probs, pred_boxes)


NameError: name 'val_dataset' is not defined